# 🔍 Tutorial 3: RAG - Query Building Codes with AI

## 🎯 What You'll Learn
- What is RAG (Retrieval Augmented Generation)
- How to query Spanish building codes (CTE)
- See actual retrieved chunks with scores
- Compare with/without RAG

---


## 🤔 What is RAG?

**Problem**: LLMs don't know about:
- Your company's documents
- Recent information
- Specific building codes

**Solution**: RAG = Retrieval + Generation

```
User Question
     ↓
🔍 Search Documents (Retrieval)
     ↓
📄 Find Relevant Chunks
     ↓
🤖 LLM + Context → Answer (Generation)
```

We have Spanish building codes (CTE):
- **CTE DB-SI**: Seguridad en caso de incendio
- **CTE DB-SUA**: Seguridad de utilización y accesibilidad

Total: ~200 pages of regulations


## 🧭 Tutorial Structure

- **Baseline**: LLM without RAG (shows generic responses)
- **Part 1**: Keyword-only search on TXT (`data/normativa/cte_db_si_ejemplo.txt`)
- **Part 2**: Hybrid retrieval + reranker using embedded PDF (`data/normativa/DBSI.pdf`)


## Setup

This notebook auto-loads the vectorstore for Part 2. If it doesn't exist, it will create it from `data/normativa/DBSI.pdf`.

**If you get Chroma instance conflicts, restart the kernel and run all cells again.**


In [5]:
import sys
from pathlib import Path
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Ensure project root is on sys.path so `src` is importable, even if kernel starts in notebooks/
ROOT = Path.cwd()
try:
    # Walk up until we find a folder containing `src`
    while ROOT != ROOT.parent and not (ROOT / 'src').exists():
        ROOT = ROOT.parent
finally:
    if (ROOT / 'src').exists() and str(ROOT) not in sys.path:
        sys.path.insert(0, str(ROOT))

from IPython.display import display, HTML
import textwrap

print("✅ Basic setup complete!")
print(f"   Project root: {ROOT}")
print(f"   Working directory: {Path.cwd()}")

# Check API key
if os.getenv("OPENAI_API_KEY"):
    print("   OpenAI API key: ✅ Found")
else:
    print("   OpenAI API key: ❌ Not found in .env")

# Try to load vectorstore for Part 2 (optional)
rag = None
try:
    from src.rag.vectorstore_manager import VectorstoreManager
    
    # Clear any existing vectorstore to avoid conflicts
    import shutil
    vectorstore_path = ROOT / "vectorstore/normativa_db"
    if vectorstore_path.exists():
        shutil.rmtree(vectorstore_path)
        print("   Cleared existing vectorstore")
    
    # Create new vectorstore
    print("   Creating vectorstore from PDFs...")
    rag = VectorstoreManager(vectorstore_path)
    rag.create_from_pdfs(ROOT / "data/normativa")
    print("   Loading vectorstore...")
    rag.load_existing()
    
    # Test it works
    print("   Testing vectorstore...")
    test_docs = rag.vectorstore.similarity_search("test", k=1)
    print(f"✅ Part 2 ready! Found {len(test_docs)} docs in vectorstore")
    
except Exception as e:
    print(f"⚠️ Part 2 (vectorstore) not available: {e}")
    print(f"   Error type: {type(e).__name__}")
    if "tenants" in str(e) or "Database error" in str(e):
        print("   💡 This is a ChromaDB schema issue. The vectorstore directory has been cleared.")
        print("   💡 Try restarting the kernel and running all cells again.")
    print("   Part 1 (TXT search) will still work")
    rag = None

print("\n🎯 Ready to start!")
print("   Baseline: LLM without RAG")
print("   Part 1: TXT keyword search (always works)")
print("   Part 2: PDF hybrid retrieval (if vectorstore loaded)")


✅ Basic setup complete!
   Project root: /Users/rauladell/Work/Servitec/aec-compliance-agent
   Working directory: /Users/rauladell/Work/Servitec/aec-compliance-agent/notebooks
   OpenAI API key: ✅ Found
   Creating vectorstore from PDFs...
Loading PDF documents...
Loading: DBSI.pdf
  - Loaded 92 pages
Total documents loaded: 92
Splitting documents into chunks...
Created 311 chunks from 92 documents
Creating embeddings and vectorstore...
Vectorstore created and saved to: /Users/rauladell/Work/Servitec/aec-compliance-agent/vectorstore/normativa_db
   Loading vectorstore...
Vectorstore loaded from: /Users/rauladell/Work/Servitec/aec-compliance-agent/vectorstore/normativa_db
   Testing vectorstore...
✅ Part 2 ready! Found 1 docs in vectorstore

🎯 Ready to start!
   Baseline: LLM without RAG
   Part 1: TXT keyword search (always works)
   Part 2: PDF hybrid retrieval (if vectorstore loaded)


/Users/rauladell/Work/Servitec/aec-compliance-agent/src/rag/vectorstore_manager.py:89: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  self.vectorstore = Chroma(


In [6]:
# Baseline: Test LLM WITHOUT RAG (shows generic responses)
print("🤖 Baseline: LLM WITHOUT RAG")
print("=" * 70)

if os.getenv("OPENAI_API_KEY"):
    try:
        from openai import OpenAI
        client = OpenAI()
        
        questions = [
            "¿Ancho mínimo de puerta de evacuación?",
            "¿Distancia máxima de evacuación en edificios?"
        ]
        
        for q in questions:
            print(f"\n❓ {q}")
            print("-" * 50)
            
            response = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": "Eres un experto en normativa de construcción española. Responde basándote en tu conocimiento general."},
                    {"role": "user", "content": q}
                ],
                temperature=0.1
            )
            
            answer = response.choices[0].message.content
            print(f"📝 Respuesta: {answer}")
            
    except Exception as e:
        print(f"⚠️ Error en LLM: {e}")
else:
    print("ℹ️ Set OPENAI_API_KEY to see LLM responses without RAG")

print("\n" + "=" * 70)
print("🔍 Now let's see how RAG improves these answers...")
print("=" * 70)


🤖 Baseline: LLM WITHOUT RAG

❓ ¿Ancho mínimo de puerta de evacuación?
--------------------------------------------------
📝 Respuesta: Según la normativa española, el ancho mínimo de una puerta de evacuación en edificios destinados a uso residencial es de 0,80 metros. Este ancho garantiza que una persona pueda salir con facilidad en caso de emergencia. Es importante cumplir con esta medida para garantizar la seguridad de los ocupantes del edificio en situaciones de evacuación.

❓ ¿Distancia máxima de evacuación en edificios?
--------------------------------------------------
📝 Respuesta: La distancia máxima de evacuación en edificios en España está regulada por el Código Técnico de la Edificación (CTE). Según el CTE, la distancia máxima de evacuación en edificios de uso residencial es de 50 metros desde cualquier punto del edificio hasta la salida de evacuación más cercana. En el caso de edificios de otros usos, como oficinas o comercios, la distancia máxima de evacuación puede variar d

In [7]:
# Part 1: Keyword-only Search on TXT
from pathlib import Path
import re

print("\n" + "#" * 70)
print("🔎 Part 1: Keyword-only Search (TXT)")
print("#" * 70)

# Use absolute path from project root
txt_path = ROOT / "data/normativa/cte_db_si_ejemplo.txt"
if not txt_path.exists():
    print(f"⚠️ TXT not found at {txt_path}. Please ensure it exists.")
else:
    raw = txt_path.read_text(encoding="utf-8", errors="ignore")

    # Very simple sectioning: split by headings like 'Sección X' or 'Capítulo X'
    sections = re.split(r"(?i)(?=\b(sección|capítulo)\s+\w+)", raw)
    # Recombine to keep the heading paired with text
    chunks = []
    for i in range(0, len(sections), 2):
        heading = sections[i].strip()
        body = sections[i+1].strip() if i + 1 < len(sections) else ""
        text = f"{heading} {body}".strip()
        if text:
            chunks.append(text)

    def keyword_rank(query: str, texts):
        q_words = [w for w in re.findall(r"\w+", query.lower()) if len(w) > 2]
        scored = []
        for t in texts:
            words = set(re.findall(r"\w+", t.lower()))
            overlap = sum(1 for w in q_words if w in words)
            density = overlap / max(len(set(q_words)), 1)
            score = overlap + 0.5 * density
            scored.append((t, score))
        return sorted(scored, key=lambda x: x[1], reverse=True)

    def render_txt_chunks(results):
        cards_html = []
        for i, (text, score) in enumerate(results, 1):
            # Best-effort parse of pseudo section heading
            m = re.search(r"(?i)\b(sección|capítulo)\s+([\w.-]+)", text)
            section = m.group(0) if m else None
            preview = textwrap.shorten(text.replace('\n', ' '), width=380, placeholder='...')
            card = f"""
            <div class='card'>
                <div class='card-title'>📄 Rank {i} — Score {score:.3f}</div>
                <div class='card-meta'>Source: cte_db_si_ejemplo.txt{f' — Section: {section}' if section else ''}</div>
                <div class='card-body'>{preview}</div>
            </div>
            """
            cards_html.append(card)
        html = f"""
        <style>
          .cards {{ display: grid; grid-template-columns: repeat(auto-fit, minmax(320px, 1fr)); gap: 12px; }}
          .card {{ border: 1px solid #e5e7eb; border-radius: 8px; padding: 12px; background: #fff; }}
          .card-title {{ font-weight: 600; margin-bottom: 6px; }}
          .card-meta {{ color: #6b7280; font-size: 12px; margin-bottom: 8px; }}
          .card-body {{ font-size: 14px; line-height: 1.4; }}
        </style>
        <div class='cards'>
          {''.join(cards_html)}
        </div>
        """
        display(HTML(html))

    keyword_questions = [
        "¿Ancho mínimo de puerta de evacuación?",
        "¿Distancia máxima de evacuación?"
    ]

    for q in keyword_questions:
        print(f"\n{'='*70}")
        print(f"❓ {q}")
        print('='*70)
        
        # Step 1: Show retrieval results
        print("🔍 STEP 1: Retrieval Results")
        ranked = keyword_rank(q, chunks)
        top = ranked[:3]
        render_txt_chunks(top)
        
        # Step 2: Show LLM response (if available)
        if os.getenv("OPENAI_API_KEY"):
            print("\n🤖 STEP 2: LLM Response")
            try:
                # Create a simple context from top results
                context = "\n\n".join([text for text, score in top])
                
                # Simple LLM call without RAG chain
                from openai import OpenAI
                client = OpenAI()
                
                response = client.chat.completions.create(
                    model="gpt-3.5-turbo",
                    messages=[
                        {"role": "system", "content": "Eres un asistente que responde ÚNICAMENTE basándote en el contexto proporcionado. NO uses conocimiento previo. Si la información no está en el contexto, di 'No se encuentra información específica en el contexto proporcionado'. Cita siempre la fuente exacta."},
                        {"role": "user", "content": f"Contexto:\n{context}\n\nPregunta: {q}\n\nResponde basándote ÚNICAMENTE en el contexto de arriba. Si no hay información suficiente, dilo claramente."}
                    ],
                    temperature=0.1
                )
                
                answer = response.choices[0].message.content
                print(f"📝 Respuesta: {answer}")
                
            except Exception as e:
                print(f"⚠️ Error en LLM: {e}")
        else:
            print("\n🤖 STEP 2: LLM Response")
            print("ℹ️ Set OPENAI_API_KEY to see LLM response")



######################################################################
🔎 Part 1: Keyword-only Search (TXT)
######################################################################

❓ ¿Ancho mínimo de puerta de evacuación?
🔍 STEP 1: Retrieval Results



🤖 STEP 2: LLM Response
📝 Respuesta: El ancho mínimo de puerta de evacuación es de 0,80 m, excepto para las puertas de evacuación de locales con ocupación inferior a 50 personas, que pueden tener un ancho libre mínimo de 0,60 m. (Fuente: CÓDIGO TÉCNICO DE LA EDIFICACIÓN, DOCUMENTO BÁSICO DB-SI: SEGURIDAD EN CASO DE INCENDIO, SECCIÓN 3: EVACUACIÓN DE OCUPANTES, 3.1 ANCHOS MÍNIMOS DE PUERTAS)

❓ ¿Distancia máxima de evacuación?
🔍 STEP 1: Retrieval Results



🤖 STEP 2: LLM Response
📝 Respuesta: La distancia máxima de evacuación desde cualquier punto de un local hasta la salida más próxima será:
- 25 m en locales de uso residencial
- 20 m en locales de uso comercial
- 15 m en locales de uso industrial.


In [8]:
# Part 2: Hybrid Retrieval + Simple Reranker (semantic + keywords)
if rag and rag.vectorstore:
    import re
    from typing import List, Tuple

    print("\n" + "#" * 70)
    print("🚀 Part 2: Hybrid Retrieval + Reranking")
    print("#" * 70)

    hybrid_questions = [
        "¿Ancho mínimo de puerta de evacuación?",
        "¿Distancia máxima de evacuación en edificios?"
    ]

    def keyword_score(text: str, query: str) -> float:
        q_words = [w for w in re.findall(r"\w+", query.lower()) if len(w) > 2]
        if not q_words:
            return 0.0
        words = set(re.findall(r"\w+", text.lower()))
        overlap = sum(1 for w in q_words if w in words)
        density = overlap / max(len(set(q_words)), 1)
        return overlap + 0.5 * density

    def render_hybrid(results: List[Tuple[object, float]]):
        cards_html = []
        for i, (doc, score) in enumerate(results, 1):
            source = doc.metadata.get('source', 'Unknown')
            page = doc.metadata.get('page', 'N/A')
            section = doc.metadata.get('section')
            section_str = f" — Section: {section}" if section else ""
            preview = textwrap.shorten(doc.page_content.replace('\n', ' '), width=380, placeholder='...')
            card = f"""
            <div class='card'>
                <div class='card-title'>📄 Rank {i} — Score {score:.3f}</div>
                <div class='card-meta'>Source: {source} — Page: {page}{section_str}</div>
                <div class='card-body'>{preview}</div>
            </div>
            """
            cards_html.append(card)
        html = f"""
        <style>
          .cards {{ display: grid; grid-template-columns: repeat(auto-fit, minmax(320px, 1fr)); gap: 12px; }}
          .card {{ border: 1px solid #e5e7eb; border-radius: 8px; padding: 12px; background: #fff; }}
          .card-title {{ font-weight: 600; margin-bottom: 6px; }}
          .card-meta {{ color: #6b7280; font-size: 12px; margin-bottom: 8px; }}
          .card-body {{ font-size: 14px; line-height: 1.4; }}
        </style>
        <div class='cards'>
          {''.join(cards_html)}
        </div>
        """
        display(HTML(html))

    for q in hybrid_questions:
        print(f"\n{'='*70}")
        print(f"❓ {q}")
        print('='*70)

        # Step 1: Show retrieval results
        print("🔍 STEP 1: Hybrid Retrieval Results")
        
        # 1) Semantic candidates with scores (fallback to inverse-rank if unavailable)
        semantic_results = []
        try:
            # Try to use vectorstore scores
            sem_with_scores = rag.vectorstore.similarity_search_with_score(q, k=8)
            semantic_results = [(doc, float(score)) for doc, score in sem_with_scores]
            # Convert distance (lower is better) to similarity-like score (higher is better)
            if semantic_results:
                max_s = max(s for _, s in semantic_results)
                min_s = min(s for _, s in semantic_results)
                denom = max(max_s - min_s, 1e-9)
                semantic_results = [(d, 1.0 - ((s - min_s) / denom)) for d, s in semantic_results]
        except Exception as e:
            print(f"⚠️ Error with vectorstore scores: {e}")
            # Fallback: inverse rank scoring
            try:
                sem_docs = rag.similarity_search(q, k=8)
                k = len(sem_docs) or 1
                semantic_results = [(d, (k - i) / k) for i, d in enumerate(sem_docs)]
            except Exception as e2:
                print(f"⚠️ Error with similarity search: {e2}")
                print("Skipping this question...")
                continue

        # 2) Keyword scoring for the same docs
        kw_scores = {id(doc): keyword_score(doc.page_content, q) for doc, _ in semantic_results}
        max_kw = max(kw_scores.values()) if kw_scores else 1.0

        # 3) Combine (70% semantic, 30% keyword)
        combined: List[Tuple[object, float]] = []
        for doc, sem_s in semantic_results:
            kw_norm = kw_scores.get(id(doc), 0.0) / max_kw if max_kw > 0 else 0.0
            final = 0.7 * sem_s + 0.3 * kw_norm
            combined.append((doc, final))

        # 4) Sort and show top results
        combined.sort(key=lambda x: x[1], reverse=True)
        top = combined[:3]
        print(f"Top {len(top)} results (hybrid + rerank):")
        render_hybrid(top)
        
        # Step 2: Show LLM response (if available)
        if os.getenv("OPENAI_API_KEY"):
            print("\n🤖 STEP 2: LLM Response")
            try:
                # Create context from top results
                context = "\n\n".join([doc.page_content for doc, score in top])
                
                # Simple LLM call
                from openai import OpenAI
                client = OpenAI()
                
                response = client.chat.completions.create(
                    model="gpt-3.5-turbo",
                    messages=[
                        {"role": "system", "content": "Eres un asistente que responde ÚNICAMENTE basándote en el contexto proporcionado. NO uses conocimiento previo. Si la información no está en el contexto, di 'No se encuentra información específica en el contexto proporcionado'. Cita siempre la fuente exacta (documento, página, sección)."},
                        {"role": "user", "content": f"Contexto:\n{context}\n\nPregunta: {q}\n\nResponde basándote ÚNICAMENTE en el contexto de arriba. Si no hay información suficiente, dilo claramente."}
                    ],
                    temperature=0.1
                )
                
                answer = response.choices[0].message.content
                print(f"📝 Respuesta: {answer}")
                
            except Exception as e:
                print(f"⚠️ Error en LLM: {e}")
        else:
            print("\n🤖 STEP 2: LLM Response")
            print("ℹ️ Set OPENAI_API_KEY to see LLM response")
else:
    print("⚠️ Part 2 skipped - vectorstore not available")
    if rag is None:
        print("   Reason: rag object is None")
    elif not hasattr(rag, 'vectorstore') or rag.vectorstore is None:
        print("   Reason: vectorstore is not initialized")
    else:
        print("   Reason: unknown")



######################################################################
🚀 Part 2: Hybrid Retrieval + Reranking
######################################################################

❓ ¿Ancho mínimo de puerta de evacuación?
🔍 STEP 1: Hybrid Retrieval Results
Top 3 results (hybrid + rerank):



🤖 STEP 2: LLM Response
📝 Respuesta: No se encuentra información específica en el contexto proporcionado.

❓ ¿Distancia máxima de evacuación en edificios?
🔍 STEP 1: Hybrid Retrieval Results
Top 3 results (hybrid + rerank):



🤖 STEP 2: LLM Response
📝 Respuesta: La distancia máxima hasta los accesos al edificio necesarios para poder llegar hasta todas sus zonas es de 30 m (Sección SI 6, documento proporcionado).


## 🎯 Summary

In this tutorial, you learned:

1. ✅ **LLM without RAG**: Shows generic responses without specific building code knowledge
2. ✅ **RAG combines retrieval + LLM generation**: Retrieval finds relevant chunks, LLM generates answers
3. ✅ **Keyword search**: Simple text matching with scores
4. ✅ **Hybrid retrieval**: Combines semantic similarity + keyword matching with reranking
5. ✅ **Visual results**: See exactly what chunks were retrieved with scores
6. ✅ **Source citations**: Always includes document, page, and section references

**Key Insight**: RAG dramatically improves answer quality by providing specific, accurate information from your documents rather than relying on the LLM's general knowledge.

**Next**: Tutorial 4 - Autonomous Agent
